In [1]:
import struct
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

# Setting

In [2]:
spark = SparkSession.builder \
            .master("spark://10.67.22.4:7077") \
            .appName("Project") \
            .config('spark.jars.packages', 'org.apache.hadoop:hadoop-common:3.2.0')\
            .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')\
            .config('spark.jars.packages', 'com.amazonaws:aws-java-sdk:1.11.375')\
            .config("spark.executor.memory", "2g")\
            .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
            .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
            .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')\
            .config('spark.hadoop.fs.s3a.access.key', '2186eedc366545058b0896dadf79d288')\
            .config('spark.hadoop.fs.s3a.secret.key', 'de147ce2e90741cdb9bae8f53ee5466e')\
            .config('spark.hadoop.fs.s3a.endpoint', 'https://cloud-areapd.pd.infn.it:5210')\
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
            .config("spark.hadoop.fs.s3a.metadatastore.impl", "org.apache.hadoop.fs.s3a.s3guard.NullMetadataStore") \
            .config("spark.hadoop.fs.s3a.path.style.access", "true") \
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled","false") \
            .config("com.amazonaws.sdk.disableCertChecking","true") \
            .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e2be70cf-b12d-46e2-b82f-b79ecae23017;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk;1.11.375 in central
	found com.amazonaws#aws-java-sdk-dlm;1.11.375 in central
	found com.amazonaws#aws-java-sdk-core;1.11.375 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.5 in central
	found org.apache.httpcomponents#httpcore;4.4.9 in central
	found commons-codec#commons-codec;1.10 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.6.7.1 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.6.0 in central
	found com.fasterxml.jackson.core#jackson-core;2.6.7 in central
	found com.fasterxml.jackson.dataformat#jackson-dataformat-cbor;2.6.7 in

	found com.amazonaws#aws-java-sdk-iot;1.11.375 in central
	found com.amazonaws#aws-java-sdk-api-gateway;1.11.375 in central
	found com.amazonaws#aws-java-sdk-acm;1.11.375 in central
	found com.amazonaws#aws-java-sdk-gamelift;1.11.375 in central
	found com.amazonaws#aws-java-sdk-dms;1.11.375 in central
	found com.amazonaws#aws-java-sdk-marketplacemeteringservice;1.11.375 in central
	found com.amazonaws#aws-java-sdk-cognitoidp;1.11.375 in central
	found com.amazonaws#aws-java-sdk-discovery;1.11.375 in central
	found com.amazonaws#aws-java-sdk-applicationautoscaling;1.11.375 in central
	found com.amazonaws#aws-java-sdk-snowball;1.11.375 in central
	found com.amazonaws#aws-java-sdk-rekognition;1.11.375 in central
	found com.amazonaws#aws-java-sdk-polly;1.11.375 in central
	found com.amazonaws#aws-java-sdk-lightsail;1.11.375 in central
	found com.amazonaws#aws-java-sdk-stepfunctions;1.11.375 in central
	found com.amazonaws#aws-java-sdk-health;1.11.375 in central
	found com.amazonaws#aws-jav

22/07/02 14:43:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [4]:
sc = spark.sparkContext
sc

<SparkContext master=spark://10.67.22.4:7077 appName=Project>

# Preprocessing data

### Load data

In [5]:
raw = spark.read.format("binaryFile").load('s3a://mapd-minidt-batch/data_*.dat')
raw = raw.select("content").rdd.flatMap(lambda el: el)

22/07/02 14:43:56 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


### Unpack data

In [6]:
def unpack(file_content):
    ppd = []
    word_counter=0
    word_size = 8                                             # size of the word in bytes
    for i in range(0, len(file_content), word_size):
        word_counter+=1
        thisInt = struct.unpack('<q', file_content[i:i+word_size])[0]
        head = (thisInt >> 61) & 0x7
        
        if head == 2:
            fpga = (thisInt >> 58) & 0x7
            chan = (thisInt >> 49) & 0x1FF
            orbit = (thisInt >> 17) & 0xFFFFFFFF
            bx = (thisInt >> 5 ) & 0xFFF
            tdc = (thisInt >> 0 ) & 0x1F
            ppd.append([fpga, chan, orbit, bx, tdc])
    return ppd

In [33]:
data = raw.flatMap(lambda file_content: unpack(file_content)).toDF(('FPGA','CHAN','ORBIT','BX','TDC'))
#data = data.persist()

### Locate chamber

In [34]:
data = data.withColumn("CHAMBER", when((data.FPGA==0) & (data.CHAN < 63), 0)\
                                    .when((data.FPGA==0) & (data.CHAN > 63) & (data.CHAN < 128), 1)\
                                    .when((data.FPGA==1) & (data.CHAN < 63), 2)\
                                    .when((data.FPGA==1) & (data.CHAN > 63) & (data.CHAN < 128), 3)\
                                    .when((data.FPGA==1) & (data.CHAN == 128), 4))
data = data.where(data.CHAMBER.isNotNull())

In [37]:
data.show(5)

+----+----+-------+----+---+-------+
|FPGA|CHAN|  ORBIT|  BX|TDC|CHAMBER|
+----+----+-------+----+---+-------+
|   1|  11|5797423| 432|  8|      2|
|   1|  12|5797423| 431|  9|      2|
|   1|  81|5797433|1889| 30|      3|
|   1|  98|5797443| 494| 18|      3|
|   0|  34|5797432|2379|  2|      0|
+----+----+-------+----+---+-------+
only showing top 5 rows



### Time information

In [38]:
data = data.withColumn('T_HIT_ns', (25 * (col('ORBIT') * 3564 + col('BX') + col('TDC')/30)))

In [40]:
data.show(5)

+----+----+-------+----+---+-------+--------------------+
|FPGA|CHAN|  ORBIT|  BX|TDC|CHAMBER|            T_HIT_ns|
+----+----+-------+----+---+-------+--------------------+
|   1|  11|5797423| 432|  8|      2|5.165504001066667E11|
|   1|  12|5797423| 431|  9|      2|   5.165504000825E11|
|   1|  81|5797433|1889| 30|      3|     5.1655132755E11|
|   1|  98|5797443| 494| 18|      3|5.165521836649999...|
|   0|  34|5797432|2379|  2|      0|5.165512506766666E11|
+----+----+-------+----+---+-------+--------------------+
only showing top 5 rows



#### Time corrections

In [41]:
data = data.withColumn("T_HIT_ns", when(data['CHAMBER']==0, data["T_HIT_ns"]+93.9)\
                                        .when(data['CHAMBER']==1, data["T_HIT_ns"]+101.4)\
                                        .when(data['CHAMBER']==2, data["T_HIT_ns"]+95.5)\
                                        .when(data['CHAMBER']==3, data["T_HIT_ns"]+92.4)\
                                        .when(data['CHAMBER']==4, data["T_HIT_ns"]))

In [43]:
data.show(5)

+----+----+-------+----+---+-------+--------------------+
|FPGA|CHAN|  ORBIT|  BX|TDC|CHAMBER|            T_HIT_ns|
+----+----+-------+----+---+-------+--------------------+
|   1|  11|5797423| 432|  8|      2|5.165504002021667E11|
|   1|  12|5797423| 431|  9|      2|    5.16550400178E11|
|   1|  81|5797433|1889| 30|      3|   5.165513276424E11|
|   1|  98|5797443| 494| 18|      3|5.165521837573999...|
|   0|  34|5797432|2379|  2|      0|5.165512507705666...|
+----+----+-------+----+---+-------+--------------------+
only showing top 5 rows

